In [6]:
from openie import StanfordOpenIE
import pathlib

In [7]:
path = pathlib.Path().resolve()
with open(str(path) + "/sample texts/SSillinois.txt") as f:
    t = f.read()



In [5]:
properties = {'openie.affinity_probability_cap': 2 / 3 }

In [6]:
t = "SS Illinois was an iron passenger-cargo steamship built by William Cramp & Sons in 1873."
with StanfordOpenIE(properties=properties) as client:
    for triple in client.annotate(t):
        print('|-', triple)

Starting server with command: java -Xmx8G -cp /Users/dr/.stanfordnlp_resources/stanford-corenlp-4.1.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-74fc3924fcb74bc2.props -preload openie
|- {'subject': 'SS Illinois', 'relation': 'was', 'object': 'iron passenger cargo steamship built'}
|- {'subject': 'SS Illinois', 'relation': 'was', 'object': 'iron passenger cargo steamship built by William Cramp'}
|- {'subject': 'SS Illinois', 'relation': 'was', 'object': 'iron passenger cargo steamship'}
|- {'subject': 'William Cramp', 'relation': 'is in', 'object': '1873'}
|- {'subject': 'SS Illinois', 'relation': 'was', 'object': 'iron passenger cargo steamship built by William Cramp in 1873'}


In [8]:
#Attempt 2 using spacy
import spacy



In [9]:
nlp = spacy.load('en_core_web_md')

In [10]:
sentences = [i for i in nlp(t).sents]
sentences

[SS Illinois was an iron passenger-cargo steamship built by William Cramp & Sons in 1873.,
 The last of a series of four Pennsylvania-class vessels, Illinois and her three sister ships—Pennsylvania, Ohio and Indiana—were the largest iron ships ever built in the United States at the time of their construction, and amongst the first to be fitted with compound steam engines.,
 They were also the first ships to challenge British dominance of the transatlantic trade since the American Civil War.
 ,
 Though soon outclassed by newer and larger vessels, Illinois was destined to enjoy a long and distinguished career, first as a transatlantic passenger liner and later as the U.S. Navy's auxiliary vessel USS Supply.,
 In the 1870s, Illinois may have been the first ship to successfully transport a shipment of fresh meat from the United States to Europe, twenty years before the introduction of refrigeration.,
 As USS Supply, the ship served in both the Spanish–American War and the First World War, 

In [11]:
def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""

  #############################################################
    for tok in nlp(sent):
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
          # check: token is a compound word or not
          if tok.dep_ == "compound":
            prefix = tok.text
            # if the previous word was also a 'compound' then add the current word to it
            if prv_tok_dep == "compound":
              prefix = prv_tok_text + " "+ tok.text

          # check: token is a modifier or not
          if tok.dep_.endswith("mod") == True:
            modifier = tok.text
            # if the previous word was also a 'compound' then add the current word to it
            if prv_tok_dep == "compound":
              modifier = prv_tok_text + " "+ tok.text

          ## chunk 3
          if tok.dep_.find("subj") == True:
            ent1 = modifier +" "+ prefix + " "+ tok.text
            prefix = ""
            modifier = ""
            prv_tok_dep = ""
            prv_tok_text = ""

          ## chunk 4
          if tok.dep_.find("obj") == True:
            ent2 = modifier +" "+ prefix +" "+ tok.text

          ## chunk 5
          # update variables
          prv_tok_dep = tok.dep_
          prv_tok_text = tok.text
      #############################################################

    return [ent1.strip(), ent2.strip()]


In [12]:

get_entities(str(sentences[0]))

['SS Illinois', 'William 1873']

In [13]:
sentences[0]

SS Illinois was an iron passenger-cargo steamship built by William Cramp & Sons in 1873.

In [14]:
from spacy.matcher import Matcher
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object
  matcher = Matcher(nlp.vocab)

  #define the pattern
  pattern = [{'DEP':'ROOT'},
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},
            {'POS':'ADJ','OP':"?"}]

  matcher.add("matching_1",[pattern])

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]]

  return(span.text)

In [15]:
get_relation(str(sentences[0]))

'was'

In [16]:
sent = str(sentences[0])
doc = nlp(sent)

# Matcher class object
matcher = Matcher(nlp.vocab)

#define the pattern
pattern = [{'DEP':'ROOT'},
        {'DEP':'prep','OP':"?"},
        {'DEP':'agent','OP':"?"},
        {'POS':'ADJ','OP':"?"}]

matcher.add("matching_1",[pattern])

matches = matcher(doc)
matches

[(11840699188806025751, 2, 3)]

In [17]:
k = len(matches) - 1
span = doc[matches[k][1]:matches[k][2]]
span.text


'was'

In [37]:
sentences[0]

SS Illinois was an iron passenger-cargo steamship built by William Cramp & Sons in 1873.